In [2]:
# from modelscope import snapshot_download
# model_dir = snapshot_download('LLM-Research/Meta-Llama-3-8B-Instruct')
# print(model_dir)

/home/ubuntu/filesystem/miniconda3/envs/llama_factory/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# print(model_dir)

/home/ubuntu/.cache/modelscope/hub/LLM-Research/Meta-Llama-3-8B-Instruct


In [2]:
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 792.8/792.8 kB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 139.5 MB/s eta 0:00:00


In [3]:
import transformers
import torch

model_id = "/workspace/instruct_finetune_llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

diff = """
diff --git a/core/filter_form_api.php b/core/filter_form_api.php
index 05d5c39e1e..7b38752fdf 100644
--- a/core/filter_form_api.php
+++ b/core/filter_form_api.php
@@ -2393,10 +2393,9 @@ function filter_form_draw_inputs( $p_filter, $p_for_screen = true, $p_static = f
 	}
 
 	if( null === $p_static_fallback_page ) {
-		$p_static_fallback_page = $_SERVER['PHP_SELF'];
-		$p_static_fallback_page = string_sanitize_url( $_SERVER['PHP_SELF'] );
+		$p_static_fallback_page = $_SERVER['SCRIPT_NAME'];
 	}
-	$t_filters_url = $p_static_fallback_page;
+	$t_filters_url = helper_mantis_url( $p_static_fallback_page );
 	$t_get_params = $_GET;
 	$t_get_params['for_screen'] = $p_for_screen;
 	$t_get_params['static'] = ON;
"""

gt_tokens = "view  _  fil  t  _  page .  phpmanage  _  filter  _  edit  _  page .  php  Fix  es"

messages = [
    {"role": "system", "content": "You are an AI assistant that explains code differences (diffs) in detail. Your task is to map each variable or function from the code to its corresponding line number and explain the changes made in the code. The explanations should focus on the purpose of the change and its impact on the code's behavior."},
    {"role": "user", "content": """
     Here is a code difference (diff) along with a set of relevant tokens from the CVE description. Please provide a detailed explanation for each variable or function mentioned in the diff, including the line number, the changes made, and their significance. 
     Diff: {diff}
     Relevant CVE tokens: {gt_tokens}
     """},]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)
print("Generated Prompt:\n", prompt)
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print()
print(outputs[0]["generated_text"][len(prompt):])

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [ ]:
import re